# Генерация датасета

In [1]:
import random
import pandas as pd


def generate_dataset(samples, features):
    object1_prefix = "obj1_"
    object2_prefix = "obj2_"
    count_of_features_per_object = features
    ds = {}
    fnamegen = feature_name_generator()
    ftypegen = feature_type_generator()
    for i in range(features):
        feature = next(fnamegen)
        type_of_feature = next(ftypegen)
        ds[f"{object1_prefix}{feature}"] = []
        ds[f"{object2_prefix}{feature}"] = []
        for i in range(samples):
            ds[f"{object1_prefix}{feature}"].append(get_random_value_by_type(type_of_feature))
            ds[f"{object2_prefix}{feature}"].append(get_random_value_by_type(type_of_feature))
    ds["collision"] = []
    for i in range(samples):
        ds["collision"].append(get_random_value_by_type("binary"))


    return ds


def feature_type_generator():
    types = ["binary", "ordinal", "numerical"]
    while True:
        for t in types:
            yield t

def get_random_value_by_type(type):
    if type == "binary":
        return random.choice([0, 1])
    elif type == "nominal":
        return random.choice(["A", "B", "C", "D"])
    elif type == "ordinal":
        return random.randint(1, 10)
    elif type == "numerical":
        return random.uniform(0, 1)

def feature_name_generator():
    i = 0
    while True: 
        i = i + 1
        yield f"feature_{i}"

    

In [2]:
count_of_samples = [30, 100, 500, 1000]
count_of_features = [4, 8, 10]

for i in count_of_samples:
    for j in count_of_features:
        pd.DataFrame(generate_dataset(i, j)).to_csv(f"out_{i}_{j}.cvs", index=False)

# Выбор алгоритма

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

models = {
    "RandomForest": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier()
}



In [4]:
df = pd.read_csv('out_1000_10.cvs')

df.head()

,obj1_feature_1,obj2_feature_1,obj1_feature_2,obj2_feature_2,obj1_feature_3,obj2_feature_3,obj1_feature_4,obj2_feature_4,obj1_feature_5,obj2_feature_5,...,obj2_feature_6,obj1_feature_7,obj2_feature_7,obj1_feature_8,obj2_feature_8,obj1_feature_9,obj2_feature_9,obj1_feature_10,obj2_feature_10,collision
0,1,0,7,5,0.873075,0.649336,1,1,5,4,...,0.190051,0,1,1,3,0.433193,0.222115,0,0,1
1,1,0,1,9,0.349867,0.422833,1,0,10,7,...,0.828490,0,1,1,6,0.107035,0.843589,0,0,1
2,1,0,2,7,0.283052,0.293933,1,0,2,7,...,0.629914,0,0,4,2,0.084224,0.753437,1,0,1
3,0,1,5,1,0.559108,0.202850,1,0,2,4,...,0.383736,1,0,5,1,0.050708,0.922973,0,0,0
4,0,1,9,6,0.835032,0.886561,0,0,4,5,...,0.470343,0,1,3,6,0.398375,0.706549,1,0,1


In [5]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X = df.drop("collision", axis=1) 
y = df["collision"]              



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Обучающая выборка: {X_train.shape[0]} строк")
print(f"Тестовая выборка: {X_test.shape[0]} строк")

results = {}

for name, model in models.items():
    print(f"Обучение модели: {name}")
    model.fit(X_train, y_train)  
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    print(f"Точность {name}: {accuracy:.2f}")
    print("-" * 50)

# Выводим результаты
print("Итоговая точность моделей:")
for name, acc in results.items():
    print(f"{name}: {acc:.2f}")



Обучающая выборка: 800 строк
Тестовая выборка: 200 строк
Обучение модели: RandomForest
Точность RandomForest: 0.55
--------------------------------------------------
Обучение модели: LogisticRegression
Точность LogisticRegression: 0.49
--------------------------------------------------
Обучение модели: SVM
Точность SVM: 0.51
--------------------------------------------------
Обучение модели: KNN
Точность KNN: 0.49
--------------------------------------------------
Итоговая точность моделей:
RandomForest: 0.55
LogisticRegression: 0.49
SVM: 0.51
KNN: 0.49


In [6]:
from timeit import Timer
from functools import partial

def forTime():
    y_pred = model.predict(X_test)
    return y_pred

for name, model in models.items():
    timed_run = Timer(partial(forTime)).timeit(number=1000)
    print(model," Время: ", timed_run)
    

RandomForestClassifier()  Время:  7.6350751999998465
LogisticRegression(max_iter=1000)  Время:  0.6944931000034558
SVC()  Время:  13.76193079999939
KNeighborsClassifier()  Время:  9.97296980000101


In [7]:
import pickle
import os

models_dir = 'saved_models_pickle'
os.makedirs(models_dir, exist_ok=True)

# Сохраняем каждую модель
for name, model in models.items():
    filename = f"{name.replace(' ', '_').lower()}.pkl"
    filepath = os.path.join(models_dir, filename)
    
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)
    print(f"Модель {name} сохранена в файл: {filepath}")


Модель RandomForest сохранена в файл: saved_models_pickle\randomforest.pkl
Модель LogisticRegression сохранена в файл: saved_models_pickle\logisticregression.pkl
Модель SVM сохранена в файл: saved_models_pickle\svm.pkl
Модель KNN сохранена в файл: saved_models_pickle\knn.pkl
